In [113]:
api_key_path = 'api_key.txt'

def load_api_key(path):
    with open(path, 'r') as file:
        return file.readline().strip()

api_key = load_api_key(api_key_path)


API Key loaded successfully!


In [119]:
import requests
import pandas as pd
import logging
def get_todays_game_ids():
    
    Client = OddsJamClient(api_key)
    Client.UseV2()
    
    # Get games for the league
    GamesResponse = Client.GetGames(league='mlb')
    Games = GamesResponse.Games
    
    # Filter games based on today's date
    games_data = [{'game_id': game.id, 'start_date': game.start_date} for game in Games]
    games_df = pd.DataFrame(games_data)
    games_df['start_date'] = pd.to_datetime(games_df['start_date'])
    #today = datetime.now().date()
    #todays_games = games_df[games_df['start_date'].dt.date == today]
    
    return games_df['game_id'].tolist()

def fetch_game_data(game_ids, markets, sportsbooks, include_player_name=True):
    URL = "https://api-external.oddsjam.com/api/v2/game-odds"
    API_KEY = api_key
    all_data = []

    for chunk in [game_ids[i:i + 5] for i in range(0, len(game_ids), 5)]:
        for sportsbook in sportsbooks:
            params = {
                'key': API_KEY,
                'sportsbook': sportsbook,
                'game_id': chunk,
                'market_name': markets
            }
            response = requests.get(URL, params=params)
            if response.status_code == 200:
                data = response.json().get('data', [])
                all_data.extend(data)
            else:
                logging.error("Error fetching data: %s - %s", response.status_code, response.text)

    rows = []
    for game_data in all_data:
        home_team = game_data.get('home_team', 'Unknown')
        away_team = game_data.get('away_team', 'Unknown')
        odds_list = game_data.get('odds', [])
            
        for item in odds_list:
            market_name = item.get('market_name', '')
            sportsbook_name = item.get('sports_book_name', sportsbook)
            if market_name not in markets:
                continue

            row = {
                'Game ID': game_data.get('id', 'Unknown'),
                "Bet Name":item.get('name',None),
                'Market Name': market_name,
                'Sportsbook': sportsbook_name,
                'line':item.get('bet_points',None),
                'Odds': item.get('price', None),
            }

            if include_player_name:
                player_name = item.get('selection', 'Unknown')
                row['Player Name'] = player_name

            rows.append(row)

    return pd.DataFrame(rows)

def fetch_player_props(game_ids):
    player_markets = [
        "Player Hits Allowed", "Player Strikeouts", "Player Home Runs",
        "Player Bases", "Player Home Runs Yes/No", "Player Outs", "Player Earned Runs"
    ]
    return fetch_game_data(game_ids, player_markets, ['Pinnacle'], include_player_name=True)

def fetch_game_props(game_ids):
    game_markets = [
        "Team Total", "1st Half Team Total", "1st Half Moneyline", "Moneyline",
        "No Runs First Inning", "1st Inning Moneyline 3-Way", "1st Half Run Line",
        "Total Runs", "1st Inning Total Runs", "1st Half Total Runs", "Run Line",
        "1st Inning Run Line", "Total Hits + Runs + Errors"
    ]
    return fetch_game_data(game_ids, game_markets, ['Pinnacle','BetMGM','Caesars'], include_player_name=False)


In [118]:
todays_ids = get_todays_game_ids()
#pdf = fetch_player_props(game_ids=todays_ids)

540d8b29-e704-403d-9926-363d3d7fcaff


In [107]:
#gdf = fetch_game_props(todays_ids)